# 0. Download seismograms

We here download a small dataset to test the package and show the main steps toward earthquake detection and location from continuous data. The dataset consists of one day of data from eight broadband seismic stations with three components. The seismic stations are located in the North Anatolian fault zone.

### Download method
This notebook uses the `mass_downloader` of `obspy` to find and download the data from the IRIS data center. The download duration can change depending on your location, do not hesitate to consider another dataset that would suit you better. A more extensive tutorial about the mass downloader [is available in obspy's documentation](https://docs.obspy.org/packages/autogen/obspy.clients.fdsn.mass_downloader.html).

### Dataset infos

| Info | Value |
|-|-|
| Number of files | 24 |
| File size (HH) | 17 MB |
| File size (BH) | 8 MB |
| Download duration | 1 minute (fastest) to 10 minutes (slowest) |
| Overall disk usage | 230 MB |

### Contents
* [Destination path](#download-destination-path)
* [Download preparation](#prepare-the-downloader)
* [Download](#download)

In [1]:
import os

from obspy import UTCDateTime
from obspy.clients.fdsn import mass_downloader

## Download destination path

By default, the package comes with a `data/` folder which can be used to store the downloaded waveforms. The waveforms downloaded there should not be pushed to a new branch if you want to contribute. We indicated the `.mseed` format in the `.gitignore` file; if you download any other data format, please update `.gitignore` accordingly. Feel free to change it to any convenient path of your choice. 

In [2]:
DIRPATH_DESTINATION = "../data/raw/"

If we decide to remove the `data/` repository in the future, this line will ensure that the path can be created.

In [3]:
os.makedirs(DIRPATH_DESTINATION, exist_ok=True)

## Prepare the downloader

We here define the geographical domain and other restrictions such as data quality and temporal coverage for downwload. We restrict the download to the North Anatolian domain thanks to the `RectangularDomain` function, and we select the data from the eight seismic stations on either channels `HH` or `BH` with available continuous data over one day (2012/07/26) within the `Restrictions` function.

In [4]:
# Geographical restrictions
domain = mass_downloader.RectangularDomain(
    minlatitude=40.60,
    maxlatitude=40.76,
    minlongitude=30.20,
    maxlongitude=30.44,
)

# Time and station restrictions
restrictions = mass_downloader.Restrictions(
    starttime=UTCDateTime("2012-07-26"),
    endtime=UTCDateTime("2012-07-27"),
    chunklength_in_sec=86400.0,
    network="YH",
    location="*",
    channel="BH*,HH*",
    station="SAUV,SPNC,DC08,DC07,DC06,DD06,DE07,DE08",
    reject_channels_with_gaps=False,
    minimum_length=0.0,
    minimum_interstation_distance_in_m=100.0,
    channel_priorities=["HH[ZNE]", "BH[ZNE]"],
)

# Downloader instance
downloader = mass_downloader.MassDownloader(providers=["IRIS"])

[2022-08-04 16:43:13,175] - obspy.clients.fdsn.mass_downloader - INFO: Initializing FDSN client(s) for IRIS.
[2022-08-04 16:43:13,663] - obspy.clients.fdsn.mass_downloader - INFO: Successfully initialized 1 client(s): IRIS.


## Download

Download the data. This can take several minutes depending on the internet connection and the machine.

In [5]:
downloader.download(
    domain,
    restrictions,
    mseed_storage=DIRPATH_DESTINATION,
    stationxml_storage=DIRPATH_DESTINATION,
)

[2022-08-04 16:43:13,752] - obspy.clients.fdsn.mass_downloader - INFO: Total acquired or preexisting stations: 0
[2022-08-04 16:43:13,755] - obspy.clients.fdsn.mass_downloader - INFO: Client 'IRIS' - Requesting reliable availability.
[2022-08-04 16:43:14,714] - obspy.clients.fdsn.mass_downloader - INFO: Client 'IRIS' - Successfully requested availability (0.96 seconds)
[2022-08-04 16:43:14,719] - obspy.clients.fdsn.mass_downloader - INFO: Client 'IRIS' - Found 8 stations (24 channels).
[2022-08-04 16:43:14,723] - obspy.clients.fdsn.mass_downloader - INFO: Client 'IRIS' - Will attempt to download data from 8 stations.
[2022-08-04 16:43:14,731] - obspy.clients.fdsn.mass_downloader - INFO: Client 'IRIS' - Status for 24 time intervals/channels before downloading: NEEDS_DOWNLOADING
[2022-08-04 16:44:53,397] - obspy.clients.fdsn.mass_downloader - INFO: Client 'IRIS' - Successfully downloaded 3 channels (of 3)
[2022-08-04 16:45:31,081] - obspy.clients.fdsn.mass_downloader - INFO: Client 'IRIS

{'IRIS': <obspy.clients.fdsn.mass_downloader.download_helpers.ClientDownloadHelper at 0x2aaae6228880>}